In [1]:
!cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
!cp /kaggle/input/backup-806/util_openbook.py .

Processing ./datasets-2.14.4-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [2]:
# installing offline dependencies
#!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
#!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
#!pip install -U /kaggle/working/sentence-transformers
#!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
#!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
#!pip install --no-index --no-deps /kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl

Processing /kaggle/input/llm-whls/transformers-4.31.0-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2


In [3]:

#from util_openbook import get_contexts, generate_openbook_output
import pickle

# get_contexts()
# generate_openbook_output()

import ctypes
libc = ctypes.CDLL("libc.so.6")

import gc
gc.collect()

101

In [4]:
import numpy as np
import pandas as pd 
from datasets import load_dataset, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata

import os
from scipy.special import softmax
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
!cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

In [6]:
USE_AMP = True
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP, init_scale=4096)

In [7]:
def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]

def get_relevant_documents_parsed(df_valid,df_chunk_size=700):
    df_chunk_size=df_chunk_size
    paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def get_relevant_documents(df_valid,df_chunk_size=800):
    #df_chunk_size=800
    
    cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def retrieval(df_valid, modified_texts):
    
    corpus_df_valid = df_valid.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    vectorizer1 = TfidfVectorizer(ngram_range=(1,6),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words)
    vectorizer1.fit(corpus_df_valid)
    vocab_df_valid = vectorizer1.get_feature_names_out()
    vectorizer = TfidfVectorizer(ngram_range=(1,6),
                                 token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_valid)
    vectorizer.fit(modified_texts[:500000])
    corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000
    top_per_chunk = 10
    top_per_query = 10

    all_chunk_top_indices = []
    all_chunk_top_values = []

    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    return articles_indices, merged_top_scores


def prepare_answering_input(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=4096,
    ):
    c_plus_q   = str(context) + ' ' + tokenizer.bos_token + ' ' + str(question)
    c_plus_q_4 = [str(c_plus_q)] * len(options)
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
    )
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    token_type_ids = tokenized_examples['token_type_ids'].unsqueeze(0)
    example_encoded = {
        "input_ids": input_ids.to('cuda'),
        "attention_mask": attention_mask.to('cuda'),
        'token_type_ids': token_type_ids.to('cuda')
    }
    return example_encoded


def prepare_answering_input2(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=4096,
    ):
    c_plus_q   = str(context) + ' ' + tokenizer.bos_token + ' ' + str(question)
    c_plus_q_4 = [str(c_plus_q)] * len(options)
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
    )
    
    #new_dict = dict()
    #new_dict['input_ids'] = torch.zeros([5, max_seq_length])

    if tokenized_examples['input_ids'].shape[1]>max_seq_length:
        #if max_seq_length==512: print(tokenized_examples['input_ids'].shape[1], 'cut to len of 512')
        start = tokenized_examples['input_ids'].shape[1] - max_seq_length
        tokenized_examples['input_ids'] = tokenized_examples['input_ids'][:,start:]
        tokenized_examples['attention_mask'] = tokenized_examples['attention_mask'][:,start:]
        tokenized_examples['token_type_ids'] = tokenized_examples['token_type_ids'][:,start:]
            
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    token_type_ids = tokenized_examples['token_type_ids'].unsqueeze(0)
    example_encoded = {
        "input_ids": input_ids.to('cuda'),
        "attention_mask": attention_mask.to('cuda'),
        'token_type_ids': token_type_ids.to('cuda')
    }
    return example_encoded


In [8]:
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']
# stop_words += ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'between', 'both', 'but', 'by', 'can', 'cannot', 'could', 'did', 'do', 'does', 'doing', 'during', 'each', 'for', 'from', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'if', 'in', 'into', 'is', 'it', 'its', 'itself', 'make', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'of', 'on', 'one', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'said', 'same', 'see', 'she', 'should', 'so', 'some', 'such', 'take', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'use', 'used', 'very', 'want', 'was', 'way', 'we', 'well', 'were', 'what', 'when', 'where', 'which', 'who', 'whom', 'why', 'will', 'with', 'would', 'you', 'your', 'yours', 'yourself', 'yourselves']
# stop_words = list(set(stop_words))

In [9]:
# from sklearn.feature_extraction import text
# stop_words2 = text.ENGLISH_STOP_WORDS
# stop_words = list(stop_words2.union(stop_words))

In [10]:
# df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv")
# df_valid2 = pd.read_csv("/kaggle/input/stem-dataset-multiple-choice-questions/stem_dataset_multiple_choice_questions_20230825_063353.csv")
# df_valid2 = df_valid2[['prompt', 'A', 'B', 'C', 'D', 'E', 'answer']]
# df_valid = pd.concat([df_valid, df_valid2])
# print(df_valid.shape)
# df_valid.head(5)

In [11]:
df_valid = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
print(df_valid.shape)

(200, 7)


In [12]:
if len(df_valid)==200:
    try:
        with open('/kaggle/input/llm-retrieved-pkls/parsed.pkl', 'rb') as f:
            retrieved_articles_parsed = pickle.load(f)
            
        print('succeeded to load pkl')
            
    except:
        retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
        with open('parsed.pkl', 'wb') as f:
            pickle.dump(retrieved_articles_parsed, f)
            
elif len(df_valid)==1200:
    try:
        with open('/kaggle/input/llm-retrieved-pkls/parsed1600.pkl', 'rb') as f:
            retrieved_articles_parsed = pickle.load(f)
        retrieved_articles_parsed = retrieved_articles_parsed[-1200:]
        print('succeeded to load pkl')
            
    except:
        seps = 600
        retrieved_articles_parsed = get_relevant_documents_parsed(df_valid, seps)
        with open('parsed1200.pkl', 'wb') as f:
            pickle.dump(retrieved_articles_parsed, f)
    
else:
    if len(df_valid)>2000:
        seps = len(df_valid)//6 + 1
    else:
        seps = 600
    retrieved_articles_parsed = get_relevant_documents_parsed(df_valid, seps)

gc.collect()

succeeded to load pkl


60

In [13]:
if len(df_valid)==200:
    try:
        with open('/kaggle/input/llm-retrieved-pkls/articles.pkl', 'rb') as f:
            retrieved_articles = pickle.load(f)
            
        print('succeeded to load pkl')
            
    except:
        retrieved_articles = get_relevant_documents(df_valid)
        with open('articles.pkl', 'wb') as f:
            pickle.dump(retrieved_articles, f)
            
elif len(df_valid)==1200:
    try:
        with open('/kaggle/input/llm-retrieved-pkls/articles1200.pkl', 'rb') as f:
            retrieved_articles = pickle.load(f)
        retrieved_articles = retrieved_articles[-1200:]
        print('succeeded to load pkl')
            
    except:
        seps = 600
        retrieved_articles = get_relevant_documents(df_valid, seps)
        with open('articles1200.pkl', 'wb') as f:
            pickle.dump(retrieved_articles, f)
            
else:
    if len(df_valid)>2000:
        seps = len(df_valid)//5 + 1
    else:
        seps = 600
    retrieved_articles = get_relevant_documents(df_valid, seps)
    
gc.collect()

succeeded to load pkl


0

In [14]:
if len(df_valid)==200:
    df_valid = df_valid[:10]
    print('saving quota to cut 200 to 10')
    
if len(df_valid)==1200:
    df_valid = pd.concat([df_valid[:10],  df_valid[210:220]])
    retrieved_articles = retrieved_articles[:10] + retrieved_articles[210:220]
    retrieved_articles_parsed = retrieved_articles_parsed[:10] + retrieved_articles_parsed[210:220]
    print('saving quota to cut 1200 to 20')

saving quota to cut 200 to 10


In [15]:
df_valid.tail()

,id,prompt,A,B,C,D,E
5,5,Which of the following statements accurately d...,Gauss's law holds only for situations involvin...,"Gauss's law holds in all cases, but it is most...","Gauss's law, which applies equally to all elec...",Gauss's law only holds for electric fields wit...,"Gauss's law, which holds for all situations, i..."
6,6,Which of the following statements accurately d...,The dimension of an object in a CW complex is ...,The dimension of an object in a CW complex is ...,The dimension of an object in a CW complex is ...,The dimension of an object in a CW complex is ...,The dimension of an object in a CW complex dep...
7,7,Which of the following statements accurately d...,The blocking temperature of an antiferromagnet...,The blocking temperature of an antiferromagnet...,The blocking temperature of an antiferromagnet...,The blocking temperature of an antiferromagnet...,The blocking temperature of an antiferromagnet...
8,8,What is the term used in astrophysics to descr...,Blueshifting,Redshifting,Reddening,Whitening,Yellowing
9,9,What is the role of axioms in a formal theory?,Basis statements called axioms form the founda...,Axioms are supplementary statements added to a...,Axioms are redundant statements that can be de...,The axioms in a theory are used for experiment...,The axioms in a formal theory are added to pro...


In [16]:
df_valid['prompt'] = df_valid['prompt'].map(str)
df_valid['A'] = df_valid['A'].map(str)
df_valid['B'] = df_valid['B'].map(str)
df_valid['C'] = df_valid['C'].map(str)
df_valid['D'] = df_valid['D'].map(str)
df_valid['E'] = df_valid['E'].map(str)

In [17]:
model_dir = "/kaggle/input/llm-cloud-2"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

preds3 = []
preds4 = []

for index in tqdm(range(df_valid.shape[0])):
    probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    
    try:
        columns = df_valid.iloc[index].values
        question = columns[1]
        options = [columns[2], columns[3], columns[4], columns[5], columns[6]]
        
        try:
            context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"

            inputs1 = prepare_answering_input(
                tokenizer=tokenizer, question=question,
                options=options, context=context1,
                )

            with torch.no_grad():
                with autocast:
                    outputs1 = model(**inputs1)    
                    losses1 = outputs1.logits[0].detach().cpu().numpy()
                    probability1 = softmax(losses1, axis=-1)
                
        except:
            print('error occurred while context1 inffering at: ', index)
            print(inputs1['input_ids'][0].shape)
            probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
            
        try:
            
            context2 = f"{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
            inputs2 = prepare_answering_input(
                tokenizer=tokenizer, question=question,
                options=options, context=context2,
                )

            with torch.no_grad():
                with autocast:
                    outputs2 = model(**inputs2)
                    losses2 = outputs2.logits[0].detach().cpu().numpy()
                    probability2 = softmax(losses2, axis=-1)
        
        except:
            print('error occurred while context2 inffering at:', index)
            print(inputs2['input_ids'][0].shape)
            probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
            
        probability_1 = probability1
        probability_2 = probability2
        
        #del inputs1, inputs2
        
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    except:
        print('unknown error occurred at:', index)
        #probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        #probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        probability_1 = probability1
        probability_2 = probability2
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    preds3.append(probability_1.reshape(1,5))
    preds4.append(probability_2.reshape(1,5))
    
_ = gc.collect()
_ = libc.malloc_trim(0)
_ = torch.cuda.empty_cache()

sub3 = np.concatenate(preds3, axis=0)
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub3.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub3.csv')

sub4 = np.concatenate(preds4, axis=0)
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub4.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub4.csv')

del preds3, preds4

_ = gc.collect()
_ = libc.malloc_trim(0)
_ = torch.cuda.empty_cache()


100%|██████████| 10/10 [00:14<00:00,  1.50s/it]


In [18]:
model_dir = "/kaggle/input/fork-of-fork-of-how-to-train-open-book-mode-b5eb2d/model_v2"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

preds5 = []
preds6 = []

for index in tqdm(range(df_valid.shape[0])):
    probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    
    try:
        columns = df_valid.iloc[index].values
        question = columns[1]
        options = [columns[2], columns[3], columns[4], columns[5], columns[6]]
        
        try:
            context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"

            inputs1 = prepare_answering_input(
                tokenizer=tokenizer, question=question,
                options=options, context=context1,
                )

            with torch.no_grad():
                #with autocast:
                outputs1 = model(**inputs1)    
                losses1 = outputs1.logits[0].detach().cpu().numpy()
                probability1 = softmax(losses1, axis=-1)

                
        except:
            print('error occurred while context1 inffering at: ', index)
            print(inputs1['input_ids'][0].shape)
            probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
            
        try:
            
            context2 = f"{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
            inputs2 = prepare_answering_input(
                tokenizer=tokenizer, question=question,
                options=options, context=context2,
                )

            with torch.no_grad():
                with autocast:
                    outputs2 = model(**inputs2)
                    losses2 = outputs2.logits[0].detach().cpu().numpy()
                    probability2 = softmax(losses2, axis=-1)
        
        except:
            print('error occurred while context2 inffering at: ', index)
            print(inputs2['input_ids'][0].shape)
            probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
            
        probability_1 = probability1
        probability_2 = probability2
        
        
        
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    except:
        print('unknown error occurred at:', index)
        #probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        #probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        probability_1 = probability1
        probability_2 = probability2
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    preds5.append(probability_1.reshape(1,5))
    preds6.append(probability_2.reshape(1,5))
    
_ = gc.collect()
_ = libc.malloc_trim(0)
_ = torch.cuda.empty_cache()

sub5 = np.concatenate(preds5, axis=0)
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub5.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub5.csv')

sub6 = np.concatenate(preds6, axis=0)
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub6.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub6.csv')

del preds5, preds6

_ = gc.collect()
_ = libc.malloc_trim(0)
_ = torch.cuda.empty_cache()


100%|██████████| 10/10 [00:13<00:00,  1.32s/it]


In [19]:

import torch.nn as nn
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#itks model 0.840
class CustomModel(nn.Module):
    def __init__(self, model_conf, *, dropout=0.2, pretrained=True):
        super().__init__()

        # Transformer
        #self.config = AutoConfig.from_pretrained(model_conf)

        self.transformer = AutoModelForMultipleChoice.from_config(model_conf)

        #self._init_weights(self.fc, self.config)

    def _init_weights(self, module, config):
        module.weight.data.normal_(mean=0.0, std=config.initializer_range)
        if module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        out = self.transformer(input_ids, attention_mask, token_type_ids=token_type_ids)
        x = out['logits'] 

        return x
    
    
MODEL_DIR = '/kaggle/input/llmse-weights-private/llm_kaggle_exp16'
CONF_PATH = MODEL_DIR + '/deberta-v3-large_config.pth'
MODEL_PATH = MODEL_DIR + '/best_model_awp_fgm.pt'
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR+'/tokenizer')
config = torch.load(CONF_PATH)
model = CustomModel(model_conf=config)
model.load_state_dict(torch.load(MODEL_PATH))
model.to(device)
model.eval()
print()

In [20]:
#predictions = []
#submit_ids = []
preds1 = []
preds2 = []

for index in tqdm(range(df_valid.shape[0])):
    probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    
    try:
        columns = df_valid.iloc[index].values

        question = columns[1]
        options = [columns[2]+columns[3]+columns[4]+columns[5]+columns[6],
                   columns[3]+columns[4]+columns[5]+columns[6]+columns[2],
                   columns[4]+columns[5]+columns[6]+columns[2]+columns[3],
                   columns[5]+columns[6]+columns[2]+columns[3]+columns[4],
                   columns[6]+columns[2]+columns[3]+columns[4]+columns[5]]
        try:
            context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"
            inputs1 = prepare_answering_input2(
                tokenizer=tokenizer, question=question,
                options=options, context=context1,
                )
            
            with torch.no_grad():
                with autocast:
                    outputs1 = model(**inputs1)    
                    losses1 = outputs1.detach().cpu().numpy()
                    probability1 = softmax(losses1, axis=-1)
                
        except:
            print('error occurred while context1 inffering at: ', index)
            print(inputs1['input_ids'][0].shape)
            probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
            
        try:
            context2 = f"{retrieved_articles_parsed[index][-4][2]}\n{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
            inputs2 = prepare_answering_input2(
                tokenizer=tokenizer, question=question,
                options=options, context=context2,
                )

            with torch.no_grad():
                with autocast:
                    outputs2 = model(**inputs2)
                    losses2 = outputs2.detach().cpu().numpy()
                    probability2 = softmax(losses2, axis=-1)
        
        except:
            print('error occurred while context2 inffering at: ', index)
            print(inputs2['input_ids'][0].shape)
            probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

        probability_1 = probability1
        probability_2 = probability2
        
        del inputs1, inputs2
        
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    except:
        print('unknown error occurred at:', index)
        #probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        #probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        probability_1 = probability1
        probability_2 = probability2
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    preds1.append(probability_1.reshape(1,5))
    preds2.append(probability_2.reshape(1,5))



100%|██████████| 10/10 [00:18<00:00,  1.88s/it]


In [21]:
_ = gc.collect()
_ = libc.malloc_trim(0)
_ = torch.cuda.empty_cache()

In [22]:
sub1 = np.concatenate(preds1, axis=0)
sub2 = np.concatenate(preds2, axis=0)

In [23]:
del preds1, preds2
_=gc.collect()
_=libc.malloc_trim(0)
_=torch.cuda.empty_cache()

In [24]:
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub1.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub1.csv')
df_prob.head()

,A_prob,B_prob,C_prob,D_prob,E_prob
0,0.004040,0.006531,0.003483,0.975098,0.010597
1,0.624512,0.123962,0.187866,0.022095,0.041046
2,0.576172,0.145386,0.086548,0.182983,0.008553
3,0.114380,0.872070,0.002420,0.002403,0.008720
4,0.125610,0.765137,0.016937,0.039093,0.053741


In [25]:
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub2.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub2.csv')
df_prob.head()

,A_prob,B_prob,C_prob,D_prob,E_prob
0,0.001100,0.000308,0.001337,0.994141,0.002831
1,0.569336,0.350342,0.040009,0.023010,0.016541
2,0.534180,0.119873,0.172363,0.156372,0.016541
3,0.144897,0.016937,0.832031,0.005047,0.001171
4,0.001772,0.000354,0.000263,0.996094,0.001457


In [26]:
del model
_=gc.collect()
_=libc.malloc_trim(0)
_=torch.cuda.empty_cache()

In [27]:

import torch.nn as nn
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#itks model 0.840
class CustomModel(nn.Module):
    def __init__(self, model_conf, *, dropout=0.2, pretrained=True):
        super().__init__()

        # Transformer
        #self.config = AutoConfig.from_pretrained(model_conf)

        self.transformer = AutoModelForMultipleChoice.from_config(model_conf)

        #self._init_weights(self.fc, self.config)

    def _init_weights(self, module, config):
        module.weight.data.normal_(mean=0.0, std=config.initializer_range)
        if module.bias is not None:
            module.bias.data.zero_()

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        out = self.transformer(input_ids, attention_mask, token_type_ids=token_type_ids)
        x = out['logits'] 

        return x
    
    
MODEL_DIR = '/kaggle/input/llmse-weights-private/llm_kaggle_exp17'
CONF_PATH = MODEL_DIR + '/deberta-v3-large_config.pth'
MODEL_PATH = MODEL_DIR + '/best_model_awp_fgm.pt'
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR+'/tokenizer')
config = torch.load(CONF_PATH)
model = CustomModel(model_conf=config)
model.load_state_dict(torch.load(MODEL_PATH))
model.to(device)
model.eval()
print()

In [28]:
#predictions = []
#submit_ids = []
preds7 = []
preds8 = []

for index in tqdm(range(df_valid.shape[0])):
    probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
    
    try:
        columns = df_valid.iloc[index].values

        question = columns[1]
        options = [columns[2]+columns[3]+columns[4]+columns[5]+columns[6],
                   columns[3]+columns[4]+columns[5]+columns[6]+columns[2],
                   columns[4]+columns[5]+columns[6]+columns[2]+columns[3],
                   columns[5]+columns[6]+columns[2]+columns[3]+columns[4],
                   columns[6]+columns[2]+columns[3]+columns[4]+columns[5]]
        try:
            context1 = f"{retrieved_articles[index][-4][2]}\n{retrieved_articles[index][-3][2]}\n{retrieved_articles[index][-2][2]}\n{retrieved_articles[index][-1][2]}"
            inputs1 = prepare_answering_input2(
                tokenizer=tokenizer, question=question,
                options=options, context=context1,
                )
            
            with torch.no_grad():
                with autocast:
                    outputs1 = model(**inputs1)    
                    losses1 = outputs1.detach().cpu().numpy()
                    probability1 = softmax(losses1, axis=-1)

        except:
            print('error occurred while context1 inffering at: ', index)
            print(inputs1['input_ids'][0].shape)
            probability1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
            
        try:
            context2 = f"{retrieved_articles_parsed[index][-4][2]}\n{retrieved_articles_parsed[index][-3][2]}\n{retrieved_articles_parsed[index][-2][2]}\n{retrieved_articles_parsed[index][-1][2]}"
            inputs2 = prepare_answering_input2(
                tokenizer=tokenizer, question=question,
                options=options, context=context2,
                )

            with torch.no_grad():
                with autocast:
                    outputs2 = model(**inputs2)
                    losses2 = outputs2.detach().cpu().numpy()
                    probability2 = softmax(losses2, axis=-1)
        
        except:
            print('error occurred while context2 inffering at: ', index)
            print(inputs2['input_ids'][0].shape)
            probability2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

        probability_1 = probability1
        probability_2 = probability2
        
        #del inputs1, inputs2
        
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    except:
        print('unknown error occurred at:', index)
        #probability_1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        #probability_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        _ = gc.collect()
        _ = libc.malloc_trim(0)
        _ = torch.cuda.empty_cache()
        
    preds7.append(probability_1.reshape(1,5))
    preds8.append(probability_2.reshape(1,5))



100%|██████████| 10/10 [00:18<00:00,  1.90s/it]


In [29]:
_ = gc.collect()
_ = libc.malloc_trim(0)
_ = torch.cuda.empty_cache()

In [30]:
sub7 = np.concatenate(preds7, axis=0)
sub8 = np.concatenate(preds8, axis=0)
del preds7, preds8
_=gc.collect()
_=libc.malloc_trim(0)
_=torch.cuda.empty_cache()

In [31]:
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub7.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub7.csv')
df_prob.head()

,A_prob,B_prob,C_prob,D_prob,E_prob
0,0.007401,0.026855,0.010117,0.919434,0.036285
1,0.917480,0.013138,0.041199,0.004086,0.023468
2,0.441650,0.203979,0.089478,0.256836,0.007755
3,0.113953,0.861816,0.009064,0.004742,0.010437
4,0.098267,0.676758,0.032349,0.114197,0.078491


In [32]:
prob_lables = ['A_prob', 'B_prob', 'C_prob', 'D_prob', 'E_prob']
df_prob = pd.DataFrame(zip(*sub8.T), index=df_valid.index, columns=prob_lables)
df_prob.to_csv('sub8.csv')
df_prob.head()

,A_prob,B_prob,C_prob,D_prob,E_prob
0,0.000515,0.000612,0.002104,0.986328,0.011505
1,0.959961,0.014717,0.013138,0.003414,0.008324
2,0.283936,0.184937,0.245117,0.254639,0.030975
3,0.234009,0.011688,0.747559,0.004318,0.002588
4,0.000919,0.000632,0.000333,0.996094,0.001893


In [33]:
del model
_=gc.collect()
_=libc.malloc_trim(0)
_=torch.cuda.empty_cache()

In [34]:
final_prob = sub1 * 1 + sub2 * 1 + sub3 * 1 + sub4 * 1 + sub5 * 1 + sub6 * 1 + sub7 * 1 + sub8 * 1 # maximise MAP@3

In [35]:
predictions_overall1 = np.argsort(-final_prob)[:,:3]
predictions_overall1[:5]

array([[3, 4, 2],
       [0, 1, 2],
       [0, 3, 2],
       [2, 1, 0],
       [3, 1, 0]])

In [36]:
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_overall1]
predictions_as_answer_letters[:5]

array([['D', 'E', 'C'],
       ['A', 'B', 'C'],
       ['A', 'D', 'C'],
       ['C', 'B', 'A'],
       ['D', 'B', 'A']], dtype='<U1')

In [37]:
predictions_as_string = df_valid['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]
predictions_as_string[:3]

['D E C', 'A B C', 'A D C']

In [38]:
submission = df_valid[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

In [39]:
submission.head(10)

,id,prediction
0,0,D E C
1,1,A B C
2,2,A D C
3,3,C B A
4,4,D B A
5,5,B C E
6,6,A C D
7,7,D B E
8,8,C B A
9,9,A E B


In [40]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current Directory:", current_directory)

Current Directory: /kaggle/working


In [41]:
!rm -rf all-paraphs-parsed-expanded
!rm -rf stem-wiki-cohere-no-emb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
